In [1]:
# Import the modules
import pandas as pd
from pathlib import Path
import hvplot.pandas
from sklearn.cluster import KMeans

In [2]:
# Read in the CSV file as a Pandas DataFrame
holiday_songs_spotify_df = pd.read_csv(
    Path("data/Holiday_Songs_Spotify.csv")
)

#C:\Users\wsabr\OneDrive\Desktop\project-4\data\Holiday_Songs_Spotify.csv
# Review the DataFrame
holiday_songs_spotify_df.head()

,Unnamed: 0,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,tempo,duration_ms,time_signature,key_mode,playlist_name,playlist_img,track_name,artist_name,album_name,album_img
0,1,00IqwkT0PZhJ86PJajRCqk,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,...,166.824,213107,3,A# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Silver Bells,Johnny Mathis,Merry Christmas,https://i.scdn.co/image/b878b9e27201163be07e74...
1,2,01h424WG38dgY34vkI3Yd0,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,...,96.013,183613,4,A major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,White Christmas,Bing Crosby,White Christmas,https://i.scdn.co/image/3bb0daf5f87a737ce67ace...
2,3,08BhfyKUXxZrnyHrDavNHP,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,...,108.043,199093,3,F# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,The Christmas Waltz,Tony Bennett,A Swingin' Christmas Featuring The Count Basie...,https://i.scdn.co/image/96aa4fb09e7fe9d38599c8...
3,4,095XSaT8I2uI6Uldj2QrSl,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,...,97.575,178680,4,A major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Stop the Cavalry,Jona Lewie,On The Other Hand There's A Fist (Remastered),https://i.scdn.co/image/50c0ea35cacbf7c5d495c7...
4,5,09b2gJR45Pyip2rx9CnXW1,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,...,119.954,203404,4,F# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,I Don't Want to Go Home for Christmas,Independent Counsel of Funk,I Don't Want to Go Home for Christmas,https://i.scdn.co/image/77eb7c17cafe55037a1ab2...


In [3]:
#drop unneeded columns
holiday_songs_features_df = holiday_songs_spotify_df.drop(['Unnamed: 0','key_mode','time_signature', 'playlist_name', 'playlist_img',
                                                           'track_name', 'artist_name','album_name', 'album_img', 'track_uri'],axis=1)
holiday_songs_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,0.1260,0.262,166.824,213107
1,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,0.4040,0.190,96.013,183613
2,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,0.0987,0.356,108.043,199093
3,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,0.0613,0.746,97.575,178680
4,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,0.2570,0.571,119.954,203404


In [4]:
#make dummies for key and mode columns
# dummies = df['YEAR'].astype(str).str.get_dummies()
#df = pd.concat([df.drop(columns='YEAR'), dummies], axis=1)

dummies_key = holiday_songs_features_df['key'].astype(str).str.get_dummies()
print(dummies_key)
dummies_mode = holiday_songs_features_df['mode'].astype(str).str.get_dummies()
print(dummies_mode)

     A  A#  B  C  C#  D  D#  E  F  F#  G  G#
0    0   1  0  0   0  0   0  0  0   0  0   0
1    1   0  0  0   0  0   0  0  0   0  0   0
2    0   0  0  0   0  0   0  0  0   1  0   0
3    1   0  0  0   0  0   0  0  0   0  0   0
4    0   0  0  0   0  0   0  0  0   1  0   0
..  ..  .. .. ..  .. ..  .. .. ..  .. ..  ..
162  0   0  0  0   0  0   0  0  1   0  0   0
163  0   0  0  0   0  0   0  0  1   0  0   0
164  0   0  0  0   0  0   0  0  0   0  1   0
165  0   0  0  0   0  0   1  0  0   0  0   0
166  0   0  0  0   0  1   0  0  0   0  0   0

[167 rows x 12 columns]
     major  minor
0        1      0
1        1      0
2        1      0
3        1      0
4        1      0
..     ...    ...
162      1      0
163      1      0
164      1      0
165      1      0
166      1      0

[167 rows x 2 columns]


In [5]:
data = [holiday_songs_features_df, dummies_key, dummies_mode]
holiday_songs_features_df_clean = pd.concat(data, axis=1)
                                            
holiday_songs_features_df_clean = holiday_songs_features_df_clean.drop(['key', 'mode'], axis=1)

holiday_songs_features_df_clean.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,C#,D,D#,E,F,F#,G,G#,major,minor
0,0.195,0.348,-10.106,0.0332,0.820000,0.000000,0.1260,0.262,166.824,213107,...,0,0,0,0,0,0,0,0,1,0
1,0.225,0.248,-15.871,0.0337,0.912000,0.000143,0.4040,0.190,96.013,183613,...,0,0,0,0,0,0,0,0,1,0
2,0.444,0.288,-11.793,0.0326,0.911000,0.000007,0.0987,0.356,108.043,199093,...,0,0,0,0,0,1,0,0,1,0
3,0.687,0.496,-4.708,0.0339,0.434000,0.001440,0.0613,0.746,97.575,178680,...,0,0,0,0,0,0,0,0,1,0
4,0.477,0.841,-5.172,0.0358,0.000165,0.097100,0.2570,0.571,119.954,203404,...,0,0,0,0,0,1,0,0,1,0


In [6]:
 # Create a a list to store inertia values and the values of k
inertia = []
k = list(range(2, 11))

In [7]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the holiday_songs_spotify DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(holiday_songs_features_df_clean)
    inertia.append(k_model.inertia_)

C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\wsabr\anaco

In [8]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,2,1.129711e+11
1,3,5.837391e+10
2,4,3.402651e+10
3,5,2.091855e+10
4,6,1.588135e+10


In [9]:
 # Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [10]:
#Using k-means with 3 clusters

In [11]:
 # Define the model with 3 clusters
model = KMeans(n_clusters=3, random_state=1)

# Fit the model
model.fit(holiday_songs_features_df_clean)

# Make predictions
k_3 = model.predict(holiday_songs_features_df_clean)

# Create a copy of the DataFrame
holiday_songs_features_clean_predictions_df = holiday_songs_features_df_clean.copy()

# Add a class column with the labels
holiday_songs_features_clean_predictions_df['features_segment'] =k_3

C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
 # Plot the clusters
holiday_songs_features_clean_predictions_df.hvplot.scatter(
    x="danceability",
    y="loudness",
    by="features_segment"
)

:NdOverlay   [features_segment]
   :Scatter   [danceability]   (loudness)